### Objetivo general del notebook

Este notebook tiene como finalidad:

1. **Revisar la estructura y contenido de las tres fuentes de datos principales del proyecto**:  
   - **JMP (WHO/UNICEF)** – archivo CSV.  
   - **World Bank WDI** – API pública.  
   - **Open-Meteo Historical Weather** – API pública.

2. **Analizar el esquema y los campos originales** de cada fuente, entendiendo:
   - Tipos de datos.  
   - Unidades.  
   - Jerarquías (país, año, residencia, servicio WASH, etc.).  
   - Relaciones potenciales entre fuentes.

3. **Realizar un data profiling inicial**, incluyendo:
   - Revisión de valores, ejemplos y formatos.    
   - Exploración de los datos del data set JMP.

4. **Inspeccionar el esquema JSON** de las APIs (World Bank y Open-Meteo) para:
   - Identificar campos relevantes.  
   - Mapear campos originales a nombres normalizados.

5. **Construir un Data Dictionary unificado**, que servirá como base para:
   - La ingesta **Bronze**.  
   - El diseño de la capa **Silver**.  
   - La futura definición del **modelo dimensional Gold**.

6. **Documentar decisiones de modelado**, incluyendo:
   - Rol de cada columna.   
   - Fuente.  
   - Relación con preguntas de negocio (Q1–Q7).


## 1. DataSet JMP – estructura del CSV

Objetivo:
- Inspeccionar columnas del dataset descargado de JMP (agua/saneamiento, etc.).
- Entender tipos de datos, valores posibles y porcentaje de nulos.

In [1]:
import pandas as pd
import numpy as np

#Cargar el archivo CSV
df= pd.read_csv('JMP_LATAM_2019_2024.csv')

df.head()

,ISO3,Country,Residence Type,Service Type,Year,Coverage,Population,Service level
0,ABW,Aruba,total,Sanitation,2019,98.72179,105707.34211,At least basic
1,ABW,Aruba,total,Hygiene,2019,98.60724,105584.69160,Basic service
2,ABW,Aruba,total,Hygiene,2019,1.22276,1309.27920,Limited service
3,ABW,Aruba,total,Sanitation,2019,0.00000,0.00000,Limited service
4,ABW,Aruba,total,Hygiene,2019,0.17000,182.02920,No handwashing facility


In [2]:
#Tamaño del tada set
df.shape

(6062, 8)

In [3]:
#Nombres de las columnas 
df.columns.to_list()

['ISO3',
 'Country',
 'Residence Type',
 'Service Type',
 'Year',
 'Coverage',
 'Population',
 'Service level']

In [4]:
# información general de tipos de datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6062 entries, 0 to 6061
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ISO3            6062 non-null   object 
 1   Country         6062 non-null   object 
 2   Residence Type  6062 non-null   object 
 3   Service Type    6062 non-null   object 
 4   Year            6062 non-null   int64  
 5   Coverage        6062 non-null   float64
 6   Population      6062 non-null   float64
 7   Service level   6062 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 379.0+ KB


In [5]:
# Tabla con: tipo, nulos, % de nulos, únicos y ejemplos
def build_column_summary(df, sample_n=3):
    summary = []
    total = len(df)
    
    for col in df.columns:
        s = df[col]
        n_missing = s.isna().sum()
        n_unique = s.nunique(dropna=True)
        examples = s.dropna().unique()[:sample_n]
        
        summary.append({
            "column": col,
            "dtype_pandas": s.dtype,
            "n_missing": int(n_missing),
            "pct_missing": round(100 * n_missing / total, 2),
            "n_unique": int(n_unique),
            "examples": examples
        })
    return pd.DataFrame(summary).sort_values("column")

col_summary = build_column_summary(df)
col_summary

,column,dtype_pandas,n_missing,pct_missing,n_unique,examples
1,Country,object,0,0.0,45,"[Aruba, Argentina, Antigua and Barbuda]"
5,Coverage,float64,0,0.0,4085,"[98.72179, 98.60724, 1.22276]"
0,ISO3,object,0,0.0,45,"[ABW, ARG, ATG]"
6,Population,float64,0,0.0,4985,"[105707.34211, 105584.6916, 1309.2792]"
2,Residence Type,object,0,0.0,3,"[total, urban, rural]"
3,Service Type,object,0,0.0,3,"[Sanitation, Hygiene, Drinking water]"
7,Service level,object,0,0.0,8,"[At least basic, Basic service, Limited service]"
4,Year,int64,0,0.0,6,"[2019, 2020, 2021]"


In [11]:
# Extraemos las combinaciones únicas de (Service Type, Service level)
pairs = (
    df[["Service Type", "Service level"]]
    .dropna()
    .drop_duplicates()
    .sort_values(["Service Type", "Service level"])
)

pairs

,Service Type,Service level
42,Drinking water,At least basic
241,Drinking water,Basic service
45,Drinking water,Limited service
246,Drinking water,Safely managed service
47,Drinking water,Surface water
50,Drinking water,Unimproved
1,Hygiene,Basic service
2,Hygiene,Limited service
4,Hygiene,No handwashing facility
0,Sanitation,At least basic


##### Diccionario JMP :combinación Service Type / Service level

| Service Type    | Service level            | Significado (JMP)                                                                                   |
|-----------------|-------------------------|--------------------------------------------------------------------------------------------------------------------|
| Drinking water  | Safely managed service  | Agua para beber de una **fuente mejorada**, ubicada en el hogar, disponible cuando se necesita y libre de contaminación fecal y química prioritaria. |
| Drinking water  | Basic service           | Agua de una fuente mejorada cuyo tiempo de recolección (ida y vuelta, incluyendo fila) es de **30 minutos o menos**. |
| Drinking water  | Limited service         | Agua de una fuente mejorada, pero el tiempo de recolección excede los **30 minutos** por viaje (ida y vuelta).    |
| Drinking water  | Unimproved              | Agua de fuentes **no mejoradas** (pozos o manantiales sin protección u otras fuentes no protegidas).              |
| Drinking water  | Surface water           | Agua tomada directamente de ríos, lagos, estanques, canales de riego u otros cuerpos de agua superficiales.       |
| Drinking water  | At least basic          | Población que usa **al menos servicio básico** de agua para beber (incluye servicio básico y gestionado de forma segura). |
| Hygiene         | Basic service           | Hogares con un lugar para lavarse las manos en casa **con agua y jabón disponibles**.                             |
| Hygiene         | Limited service         | Hogares con un lugar para lavarse las manos, pero falta agua, jabón o ambos.                                      |
| Hygiene         | No handwashing facility | Hogares **sin instalación** para lavado de manos en el domicilio.                                                 |
| Sanitation      | Safely managed service  | Uso de instalaciones de saneamiento mejoradas, **no compartidas**, donde las excretas se contienen y se gestionan de forma segura. |
| Sanitation      | Basic service           | Uso de instalaciones de saneamiento **mejoradas y no compartidas**, sin requerir evidencia de gestión segura de excretas. |
| Sanitation      | Limited service         | Uso de instalaciones de saneamiento mejoradas, **compartidas** entre dos o más hogares.                           |
| Sanitation      | Unimproved              | Uso de instalaciones **no mejoradas** (letrinas sin losa, letrinas colgantes, cubos, etc.).                       |
| Sanitation      | Open defecation         | Defecación al aire libre: campos, bosques, cuerpos de agua u otros espacios abiertos.                             |
| Sanitation      | At least basic          | Población que utiliza **al menos saneamiento básico** (instalaciones mejoradas no compartidas, incluyendo las gestionadas de forma segura). |

| Pregunta           | Descripción                                      | Service Type   | Service level                                   | Residence Type             | Métrica            |
|--------------------------|--------------------------------------------------|----------------|------------------------------------------------|----------------------------|--------------------|
| p1_agua_segura_rur_urb   | Agua Segura (Urbano/Rural)                       | Drinking water | Safely managed service                          | rural, urban         | Coverage           |
| p2_movilidad_limitada    | Agua Limitada (>30 min)                          | Drinking water | Limited service                                 | total                  | Coverage o Population para cuantificar el número de afectados           |
| p3_4_riesgo_sanitario    | Población en Alto Riesgo Sanitario         | Sanitation     | Limited service, Unimproved, Open defecation | total            | Suma de Coverage   |
| p5_agua_segura_brecha    | Brecha Agua Segura (Rural vs. Urbano)           | Drinking water | At least basic                          | rural, urban        | Coverage           |
| p6_agua_segura_total     | Agua Segura Total (Correlación PIB)             | Drinking water | Safely managed service                          | total                  | Coverage           |
| p7_saneamiento_seg_total | Saneamiento Seguro Total (Correlación Agua)     | Drinking water y Sanitation     | Safely managed service                          | total                 | Coverage           |


## 2. API World Bank (WDI) – estructura del JSON

En esta sección sólo se inspecciona la estructura del JSON devuelto por la API de World Bank para un punto geográfico de ejemplo.

**Objetivo:**
- Ver cómo viene organizada la respuesta.
- Identificar los campos que usaremos más adelante en el diccionario de datos.

In [8]:
import requests, json

# País e indicador de ejemplo
country = "MEX"
indicator = "SH.DYN.MORT"   # Mortalidad en menores de 5 años por cada 1,000 nacidos vivos

# Año de ejemplo 
year = 2020

url = (
    f"https://api.worldbank.org/v2/country/{country}/indicator/{indicator}"
    f"?date={year}"
    "&format=json&per_page=20000"
)

resp = requests.get(url)
data_json = resp.json()

print(f"Respuesta JSON de la API WDI para {country} ({indicator}), año {year}:\n")
print(json.dumps(data_json, indent=2, ensure_ascii=False))

Respuesta JSON de la API WDI para MEX (SH.DYN.MORT), año 2020:

[
  {
    "page": 1,
    "pages": 1,
    "per_page": 20000,
    "total": 1,
    "sourceid": "2",
    "lastupdated": "2025-10-07"
  },
  [
    {
      "indicator": {
        "id": "SH.DYN.MORT",
        "value": "Mortality rate, under-5 (per 1,000 live births)"
      },
      "country": {
        "id": "MX",
        "value": "Mexico"
      },
      "countryiso3code": "MEX",
      "date": "2020",
      "value": 13.8,
      "unit": "",
      "obs_status": "",
      "decimal": 0
    }
  ]
]


## 3. API Open-Meteo – estructura del JSON

En esta sección sólo se inspecciona la estructura del JSON devuelto por la API de Open-Meteo para un punto geográfico de ejemplo.

**Objetivo:**
- Ver cómo viene organizada la respuesta.
- Identificar los campos que usaremos más adelante en el diccionario de datos.

In [9]:
# Punto de referencia para México (ejemplo: Ciudad de México)
country_iso3 = "MEX"
latitude = 19.4326
longitude = -99.1332

# Fecha de ejemplo 
date = "2020-01-01"

url = (
    "https://archive-api.open-meteo.com/v1/archive"
    f"?latitude={latitude}"
    f"&longitude={longitude}"
    "&daily=precipitation_sum"        
    f"&start_date={date}"
    f"&end_date={date}"               
    "&timezone=auto"
)

resp = requests.get(url)
weather_json = resp.json()

print(
    "Respuesta JSON de la API Open-Meteo\n"
    f"País (referencia): {country_iso3}, coordenadas ({latitude}, {longitude}), fecha {date}:\n"
)
print(json.dumps(weather_json, indent=2, ensure_ascii=False))

Respuesta JSON de la API Open-Meteo
País (referencia): MEX, coordenadas (19.4326, -99.1332), fecha 2020-01-01:

{
  "latitude": 19.437609,
  "longitude": -99.10715,
  "generationtime_ms": 2.3031234741210938,
  "utc_offset_seconds": -21600,
  "timezone": "America/Mexico_City",
  "timezone_abbreviation": "GMT-6",
  "elevation": 2230.0,
  "daily_units": {
    "time": "iso8601",
    "precipitation_sum": "mm"
  },
  "daily": {
    "time": [
      "2020-01-01"
    ],
    "precipitation_sum": [
      0.2
    ]
  }
}


## 4. Diccionario del Dataset (Data Dictionary)

Este diccionario consolida la documentación técnica de las columnas provenientes de las tres fuentes del proyecto:

- **JMP (WHO/UNICEF)**: Indicadores WASH (agua, saneamiento e higiene).
- **World Bank WDI**: Indicadores económicos y sociales.
- **Open-Meteo**: Datos climáticos históricos (precipitación diaria).

**Objetivo**:

1. Estandarizar nombres de columnas mediante column_normalized, facilitando la integración entre fuentes heterogéneas.
2. Documentar significado, origen, ejemplos y uso analítico de cada campo.
3. Registrar los tipos esperados en el modelo analítico (dtype_logical).
4. Mantener la trazabilidad entre columnas y preguntas de negocio, asegurando que cada variable aporta valor analítico claro.

In [10]:
data_dictionary = pd.DataFrame([
    # =========================
    # Dataset JMP (WASH)
    # =========================
    {
        "source_system": "JMP_WHO_UNICEF",
        "source_format": "CSV",
        "business_questions": "Q1,Q2,Q3,Q4,Q5,Q7",
        "column_original": "Country",
        "column_normalized": "country_name",          
        "dtype_logical": "STRING",
        "description": "Nombre del país.",
        "examples": "Aruba, Argentina, Antigua and Barbuda",
        "notes": "Dimensión geográfica. Se alinea con ISO3 para vincularse con World Bank y Open-Meteo."
    },
    {
        "source_system": "JMP_WHO_UNICEF",
        "source_format": "CSV",
        "business_questions": "Q1,Q2,Q3,Q4,Q5,Q6,Q7",
        "column_original": "ISO3",
        "column_normalized": "country_iso3",
        "dtype_logical": "CHAR(3)",
        "description": "Código ISO 3166-1 alfa-3 del país.",
        "examples": "ABW, ARG, ATG, BES, BHS",
        "notes": "Clave estándar para unir JMP, World Bank y datos climáticos por país."
    },
    {
        "source_system": "JMP_WHO_UNICEF",
        "source_format": "CSV",
        "business_questions": "Q1,Q2,Q5",
        "column_original": "Residence Type",
        "column_normalized": "residence_type",
        "dtype_logical": "STRING",
        "description": "Tipo de residencia: total / urbano / rural.",
        "examples": "total, urban, rural",
        "notes": "Permite analizar brechas urbano–rural y comparar patrones de acceso a servicios."
    },
    {
        "source_system": "JMP_WHO_UNICEF",
        "source_format": "CSV",
        "business_questions": "Q1,Q2,Q3,Q4,Q5,Q7",
        "column_original": "Service Type",
        "column_normalized": "service_type",
        "dtype_logical": "STRING",
        "description": "Dominio de servicio WASH reportado.",
        "examples": "Sanitation, Hygiene, Drinking water",
        "notes": "Candidata a dimensión pequeña (dim_service_type)."
    },
    {
        "source_system": "JMP_WHO_UNICEF",
        "source_format": "CSV",
        "business_questions": "Q1,Q2,Q3,Q4,Q5,Q7",
        "column_original": "Service level",
        "column_normalized": "service_level",
        "dtype_logical": "STRING",
        "description": "Nivel de servicio dentro del dominio WASH.",
        "examples": "At least basic, Basic service, Limited",
        "notes": "Dimensión de nivel de servicio. Incluye categorías como safely managed, basic, limited, unimproved, open defecation."
    },
    {
        "source_system": "JMP_WHO_UNICEF",
        "source_format": "CSV",
        "business_questions": "Q1,Q2,Q3,Q4,Q5,Q6,Q7",
        "column_original": "Coverage",
        "column_normalized": "coverage_pct",
        "dtype_logical": "DECIMAL(5,2)",
        "description": "Porcentaje de población con el nivel de servicio indicado.",
        "examples": "98.72, 0.17, 45.30",
        "notes": "Unidad: porcentaje 0–100. Indicador central para medir cobertura WASH."
    },
    {
        "source_system": "JMP_WHO_UNICEF",
        "source_format": "CSV",
        "business_questions": "Q1,Q2,Q3,Q4,Q5,Q7",
        "column_original": "Population",
        "column_normalized": "population",
        "dtype_logical": "BIGINT",
        "description": "Población asociada al registro de cobertura.",
        "examples": "105707, 105584, 1309",
        "notes": "Se recomienda redondear a entero en capas posteriores. Permite pasar de porcentaje a conteo absoluto."
    },
    {
        "source_system": "JMP_WHO_UNICEF",
        "source_format": "CSV",
        "business_questions": "Q1,Q2,Q3,Q4,Q5,Q6,Q7",
        "column_original": "Year",
        "column_normalized": "year",
        "dtype_logical": "INT",
        "description": "Año de referencia del dato de cobertura.",
        "examples": "2019, 2020, 2021, 2022, 2023, 2024",
        "notes": "Se usará para construir dim_date/dim_year y analizar tendencias temporales."
    },

    # =========================
    # World Bank API (WDI)
    # =========================
    {
        "source_system": "WORLD_BANK_WDI",
        "source_format": "JSON",
        "business_questions": "Q1,Q6",
        "column_original": "NY.GDP.PCAP.CD",
        "column_normalized": "gdp_per_capita_usd_current",
        "dtype_logical": "DECIMAL(18,2)",
        "description": "PIB per cápita en dólares estadounidenses corrientes.",
        "examples": "1578.34, 10345.78, 45231.10",
        "notes": "Indicador WDI NY.GDP.PCAP.CD. Fact económico clave para analizar relación entre desarrollo y acceso al agua."
    },
    {
        "source_system": "WORLD_BANK_WDI",
        "source_format": "JSON",
        "business_questions": "Q2,Q3,Q4,Q6",
        "column_original": "SI.POV.DDAY",
        "column_normalized": "poverty_headcount",
        "dtype_logical": "DECIMAL(5,2)",
        "description": "Porcentaje de población viviendo con menos de 3 USD/día (umbral de pobreza extrema).",
        "examples": "2.50, 15.30, 48.70",
        "notes": "Indicador WDI SI.POV.DDAY (PPP 2021). Permite medir vulnerabilidad socioeconómica."
    },
    {
        "source_system": "WORLD_BANK_WDI",
        "source_format": "JSON",
        "business_questions": "Q4",
        "column_original": "SH.DYN.MORT",
        "column_normalized": "under5_mortality_per_1000",
        "dtype_logical": "DECIMAL(6,2)",
        "description": "Mortalidad en menores de 5 años por cada 1,000 nacidos vivos.",
        "examples": "8.20, 35.60, 72.10",
        "notes": "Indicador WDI SH.DYN.MORT. Se usa como proxy de riesgo sanitario asociado al entorno y condiciones de saneamiento."
    },
    {
        "source_system": "WORLD_BANK_WDI",
        "source_format": "JSON",
        "business_questions": "Q1,Q5,Q6,Q7",
        "column_original": "SH.H2O.BASW.ZS",
        "column_normalized": "basic_drinking_water_pct",
        "dtype_logical": "DECIMAL(5,2)",
        "description": "Porcentaje de población que usa al menos servicios básicos de agua potable.",
        "examples": "65.40, 89.75, 99.10",
        "notes": "Indicador WDI SH.H2O.BASW.ZS. Sirve para validar/contrastar datos de JMP y para análisis país-año agregados."
    },
    {
        "source_system": "WORLD_BANK_WDI",
        "source_format": "JSON",
        "business_questions": "Q3,Q4,Q7",
        "column_original": "SH.STA.BASS.ZS",
        "column_normalized": "basic_sanitation_pct",
        "dtype_logical": "DECIMAL(5,2)",
        "description": "Porcentaje de población con al menos servicios básicos de saneamiento.",
        "examples": "40.20, 72.50, 95.10",
        "notes": "Indicador WDI SH.STA.BASS.ZS. Mide cobertura mínima de saneamiento; complementa a JMP para análisis de vulnerabilidad."
    },
    {
        "source_system": "WORLD_BANK_WDI",
        "source_format": "JSON",
        "business_questions": "Q3,Q4,Q7",
        "column_original": "SH.STA.SMSS.ZS",
        "column_normalized": "safely_managed_sanitation_pct",
        "dtype_logical": "DECIMAL(5,2)",
        "description": "Porcentaje de población con saneamiento gestionado de forma segura (safely managed).",
        "examples": "10.50, 45.70, 80.25",
        "notes": "Indicador WDI SH.STA.SMSS.ZS. Nivel más alto de la ladder de saneamiento; complementa a JMP para análisis de vulnerabilidad, clave para identificar brechas entre agua y saneamiento."
    },

    # =========================
    # Open-Meteo API (clima)
    # =========================
    {
        "source_system": "OPEN_METEO",
        "source_format": "JSON",
        "business_questions": "Q1,Q3,Q4",
        "column_original": "daily.precipitation_sum",
        "column_normalized": "precipitation_sum_mm_day",
        "dtype_logical": "DECIMAL(10,3)",
        "description": "Precipitación diaria acumulada en milímetros para cada fecha y ubicación.",
        "examples": "0.000, 2.345, 15.800",
        "notes": "Historical Weather API de Open-Meteo (parámetro daily=precipitation_sum). Base para agregar a nivel mensual/anual por país."
    },
])

data_dictionary

,source_system,source_format,business_questions,column_original,column_normalized,dtype_logical,description,examples,notes
0,JMP_WHO_UNICEF,CSV,"Q1,Q2,Q3,Q4,Q5,Q7",Country,country_name,STRING,Nombre del país.,"Aruba, Argentina, Antigua and Barbuda",Dimensión geográfica. Se alinea con ISO3 para ...
1,JMP_WHO_UNICEF,CSV,"Q1,Q2,Q3,Q4,Q5,Q6,Q7",ISO3,country_iso3,CHAR(3),Código ISO 3166-1 alfa-3 del país.,"ABW, ARG, ATG, BES, BHS","Clave estándar para unir JMP, World Bank y dat..."
2,JMP_WHO_UNICEF,CSV,"Q1,Q2,Q5",Residence Type,residence_type,STRING,Tipo de residencia: total / urbano / rural.,"total, urban, rural",Permite analizar brechas urbano–rural y compar...
3,JMP_WHO_UNICEF,CSV,"Q1,Q2,Q3,Q4,Q5,Q7",Service Type,service_type,STRING,Dominio de servicio WASH reportado.,"Sanitation, Hygiene, Drinking water",Candidata a dimensión pequeña (dim_service_type).
4,JMP_WHO_UNICEF,CSV,"Q1,Q2,Q3,Q4,Q5,Q7",Service level,service_level,STRING,Nivel de servicio dentro del dominio WASH.,"At least basic, Basic service, Limited",Dimensión de nivel de servicio. Incluye catego...
5,JMP_WHO_UNICEF,CSV,"Q1,Q2,Q3,Q4,Q5,Q6,Q7",Coverage,coverage_pct,"DECIMAL(5,2)",Porcentaje de población con el nivel de servic...,"98.72, 0.17, 45.30",Unidad: porcentaje 0–100. Indicador central pa...
6,JMP_WHO_UNICEF,CSV,"Q1,Q2,Q3,Q4,Q5,Q7",Population,population,BIGINT,Población asociada al registro de cobertura.,"105707, 105584, 1309",Se recomienda redondear a entero en capas post...
7,JMP_WHO_UNICEF,CSV,"Q1,Q2,Q3,Q4,Q5,Q6,Q7",Year,year,INT,Año de referencia del dato de cobertura.,"2019, 2020, 2021, 2022, 2023, 2024",Se usará para construir dim_date/dim_year y an...
8,WORLD_BANK_WDI,JSON,"Q1,Q6",NY.GDP.PCAP.CD,gdp_per_capita_usd_current,"DECIMAL(18,2)",PIB per cápita en dólares estadounidenses corr...,"1578.34, 10345.78, 45231.10",Indicador WDI NY.GDP.PCAP.CD. Fact económico c...
9,WORLD_BANK_WDI,JSON,"Q2,Q3,Q4,Q6",SI.POV.DDAY,poverty_headcount,"DECIMAL(5,2)",Porcentaje de población viviendo con menos de ...,"2.50, 15.30, 48.70",Indicador WDI SI.POV.DDAY (PPP 2021). Permite ...
